In [ ]:
!pip install cryptography

In [ ]:
import os
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

# --- Configuration ---
PRIVATE_KEY_FILE = "private_key.pem"
PUBLIC_KEY_FILE = "public_key.pem"
PASSWORD = b"my-super-secret-password" # Use a strong password in a real app!
# ---

def generate_and_save_keys():
    """
    Generates a new RSA private key and saves it to a file,
    then extracts and saves the public key.
    """
    print("Generating new private/public key pair...")

    # 1. Generate the private key
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
    )

    # 2. Serialize the private key in PEM format, encrypted with a password
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.BestAvailableEncryption(PASSWORD)
    )

    # 3. Save the private key to a file
    with open(PRIVATE_KEY_FILE, "wb") as f:
        f.write(private_pem)
    print(f"Success! Private key saved to: {PRIVATE_KEY_FILE}")
    print("WARNING: Keep this file safe and secret!\n")

    # 4. Generate the public key
    public_key = private_key.public_key()

    # 5. Serialize the public key in PEM format
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    # 6. Save the public key to a file
    with open(PUBLIC_KEY_FILE, "wb") as f:
        f.write(public_pem)
    print(f"Success! Public key saved to: {PUBLIC_KEY_FILE}")
    print("You can share this public key with anyone.\n")


if __name__ == "__main__":
    # Check if keys already exist to avoid overwriting
    if os.path.exists(PRIVATE_KEY_FILE) or os.path.exists(PUBLIC_KEY_FILE):
        print("Key files already exist. Using existing keys.")
        print("If you want new keys, delete the .pem files and run this cell again.")
    else:
        generate_and_save_keys()


Generating new private/public key pair...
Success! Private key saved to: private_key.pem

Success! Public key saved to: public_key.pem
You can share this public key with anyone.



In [ ]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.backends import default_backend
import sys
import os # Import os to use for file paths
import base64 # Import base64 for encoding/decoding

# --- Configuration (must match Cell 2) ---
PRIVATE_KEY_FILE = "private_key.pem"
PASSWORD = "my-super-secret-password" # The password you set in Cell 2
# ---

def load_private_key(filename, password):
    """Loads a password-protected private key from a PEM file."""
    try:
        with open(filename, "rb") as key_file:
            private_key = serialization.load_pem_private_key(
                key_file.read(),
                password=password.encode(),
                backend=default_backend()
            )
        return private_key
    except FileNotFoundError:
        print(f"Error: Private key file not found at {filename}")
        return None
    except (TypeError, ValueError):
        print("Error: Incorrect password for private key.")
        return None
    except Exception as e:
        print(f"An error occurred loading the private key: {e}")
        return None

def load_public_key(filename):
    """Loads a public key from a PEM file."""
    try:
        with open(filename, "rb") as key_file:
            public_key = serialization.load_pem_public_key(
                key_file.read(),
                backend=default_backend()
            )
        return public_key
    except FileNotFoundError:
        print(f"Error: Public key file not found at {filename}")
        return None
    except Exception as e:
        print(f"An error occurred loading the public key: {e}")
        return None

def encrypt_message(message, public_key):
    """Encrypts a message using a recipient's public key."""
    encrypted_message = public_key.encrypt(
        message.encode(), # Convert string to bytes
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_message

def decrypt_message(encrypted_message, private_key):
    """Decrypts a message using your private key."""
    try:
        decrypted_message = private_key.decrypt(
            encrypted_message,
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        return decrypted_message.decode() # Convert bytes back to string
    except Exception as e:
        print(f"Decryption failed. This message may be corrupted or not for you. Error: {e}")
        return None

# --- Main Application Loop ---

def main_menu():
    """Displays the main menu and runs the application."""
    print("\n--- Secure Messenger v1.0 ---")
    while True:
        print("\nWhat would you like to do?")
        print("  1. Encrypt a message for someone")
        print("  2. Decrypt a message from someone")
        print("  3. Exit")
        choice = input("Enter your choice (1, 2, or 3): ").strip()

        if choice == '1':
            action_encrypt()
        elif choice == '2':
            action_decrypt()
        elif choice == '3':
            print("Exiting. Stay secure!")
            break
        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

def action_encrypt():
    """Walks user through encrypting a message."""
    print("\n[Encrypt Message]")
    pub_key_path = input("Enter path to recipient's public key (e.g., public_key.pem): ")
    public_key = load_public_key(pub_key_path)

    if public_key:
        message = input("Enter the secret message to encrypt:\n")
        encrypted_data = encrypt_message(message, public_key)

        # In a real app, you'd send this data. Here we'll print it
        # in a format (Base64) that's safe to copy/paste.
        encrypted_text = base64.b64encode(encrypted_data).decode('utf-8')
        print("\n--- ENCRYPTED MESSAGE (Share this text) ---")
        print(encrypted_text)
        print("------------------------------------------")

def action_decrypt():
    """Walks user through decrypting a message."""
    print("\n[Decrypt Message]")
    # We already know the private key path from the config
    priv_key_path = PRIVATE_KEY_FILE

    # Check if the private key file exists first
    if not os.path.exists(priv_key_path):
        print(f"Error: Private key file not found at {priv_key_path}")
        print("Please make sure you've run Cell 2 to generate keys.")
        return

    password = input("Enter your private key password: ")

    private_key = load_private_key(priv_key_path, password)

    if private_key:
        encrypted_text = input("Paste the encrypted (Base64) message:\n")

        try:
            encrypted_data = base64.b64decode(encrypted_text)
        except Exception as e:
            print(f"Error: Invalid Base64 data. {e}")
            return

        decrypted_message = decrypt_message(encrypted_data, private_key)

        if decrypted_message:
            print("\n--- DECRYPTED MESSAGE ---")
            print(decrypted_message)
            print("-------------------------")

if __name__ == "__main__":
    print("Welcome! Make sure you have run Cell 2 to generate keys.")
    main_menu()


Welcome! Make sure you have run Cell 2 to generate keys.

--- Secure Messenger v1.0 ---

What would you like to do?
  1. Encrypt a message for someone
  2. Decrypt a message from someone
  3. Exit
Enter your choice (1, 2, or 3): 1

[Encrypt Message]
Enter path to recipient's public key (e.g., public_key.pem): public_key.pem
Enter the secret message to encrypt:
The eagle flies at dawn

--- ENCRYPTED MESSAGE (Share this text) ---
sjVMtuSbwk3BleronzD+h+vvfdjyQbG1secxAXpLsK/QrhoIrTnrOfDCxXpUBBjnbfhgQrbvgmNUokl6lvnEaeAzdsqatq1QQhaOwJNNVcDmgxYWfn7E7j/qNTGHtKW0xprJpql6JBdF5o1OcCK97uzgRukGyf3PWK0cuWLXmLP+UtThcWIVJh8H7/Fe4RA6IDMVFIBe3ZQTEDVg5YTcPo7nsnTb+ToWDAecsv1IFsNQzVA0RYrPwoPlxxkxXOhnqDWoh4q2PKHjeXbP0XrLef9HqapjN1GtjiSxk2OjzgS5oMpVavEFyebWSj5oRF8r/2D7GL1EUXxHNmJqm03wjA==
------------------------------------------

What would you like to do?
  1. Encrypt a message for someone
  2. Decrypt a message from someone
  3. Exit
Enter your choice (1, 2, or 3): 2

[Decrypt Message]
Enter your private